### 실습
- 벡터db를 이용한 검색 시스템 만들기
- llm 모델을 활용해서 자연어 검색이 매끄럽게 동작하도록 할 것
- ui(gui, cli 구분없음) 구현을 할 것

In [1]:
# Import Package
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import CharacterTextSplitter
import openai

# OpenAI API 설정
openai.api_key = ''

# HuggingFace 모델 임베딩 로드
embeddings = HuggingFaceBgeEmbeddings()

# Load Data
documents = TextLoader("./소설.md", encoding='utf8').load()

# Text Split
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# Create DB
db = FAISS.from_documents(docs, embeddings)

# LLM을 통한 쿼리 보완 함수
def improve_query_with_gpt4(query):
    prompt = f"사용자가 입력한 검색어 '{query}'를 더 구체적이고 관련성 높은 쿼리로 보완해줘."
    
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that improves search queries."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=50,
        temperature=0.5
    )
    
    improved_query = response.choices[0].message["content"].strip()
    return improved_query

# 검색 실행 함수
def run_cli_search():
    while True:
        # 사용자 입력 받기
        query = input("검색어를 입력하세요 (종료하려면 'exit' 입력): ")
        if query.lower() == "exit":
            print("검색을 종료합니다.")
            break

        # GPT-4를 사용해 쿼리 보완
        improved_query = improve_query_with_gpt4(query)
        print(f"\nGPT-4가 보완한 쿼리: {improved_query}\n")

        # FAISS를 통한 유사도 검색
        answer = db.similarity_search(improved_query)

        # 결과 출력
        print("검색 결과:")
        print(answer[0].page_content)

# CLI 실행
if __name__ == "__main__":
    run_cli_search()


d:\workspace\Chunjae_Bigdata_9th\9.4~9.23\.conda\lib\site-packages\IPython\core\interactiveshell.py:3508: LangChainDeprecationWarning: Default values for HuggingFaceBgeEmbeddings.model_name were deprecated in LangChain 0.2.5 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceBgeEmbeddings constructor instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
d:\workspace\Chunjae_Bigdata_9th\9.4~9.23\.conda\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange



GPT-4가 보완한 쿼리: '콩쥐팥쥐'에 대한 검색 쿼리를 더 구체적이고 관련성 높게 보완하면 다음과 같습니다:

1. "콩쥐팥쥐 이야기 줄거리 및

검색 결과:
춘향전은 이몽룡과 성춘향의 사랑 이야기입니다. 이몽룡은 양반집 도령이고, 춘향은 기생의 딸로 신분 차이에도 불구하고 두 사람은 사랑에 빠집니다. 그러나 이몽룡이 과거 시험을 보러 떠난 사이, 변사또라는 탐관오리가 춘향에게 수청을 강요합니다. 춘향은 끝까지 이몽룡만을 기다리며 그 부당한 요구를 거절합니다. 결국 이몽룡은 과거에 급제해 암행어사로 돌아와 춘향을 구하고, 두 사람은 행복하게 결혼하게 됩니다.

# 적벽대전
검색을 종료합니다.
